In [115]:
!git clone github.com/JuanIsernGhosn/coral-classification

hint: You've added another git repository inside your current repository.
hint: Clones of the outer repository will not contain the contents of
hint: the embedded repository and will not know how to obtain it.
hint: If you meant to add a submodule, use:
hint: 
hint: 	git submodule add <url> coral-classification-deep-learning
hint: 
hint: If you added this path by mistake, you can remove it from the
hint: index with:
hint: 
hint: 	git rm --cached coral-classification-deep-learning
hint: 
hint: See "git help submodule" for more information.
/content


In [0]:
experiment = "coral-classification-deep-learning/experiments/vgg16_Keras_1/"

Carga de librerías


In [0]:
import numpy as np 
import os
import cv2
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense, Conv2D, MaxPooling2D, ZeroPadding2D, GlobalAveragePooling2D
from keras.layers.core import Flatten, Dropout
from keras.preprocessing.image import ImageDataGenerator
from keras import backend as K
from keras.models import Model
import keras
import tensorflow as tf
from keras.optimizers import SGD
from keras.applications import vgg16
from keras.callbacks import EarlyStopping 
from natsort import natsorted, ns
from keras.utils import to_categorical

Set global parameters

In [0]:
# Fijar las semillas
seed = 2032 # Semilla del numpy
tf.set_random_seed(seed)# Fijar semilla del keras/tensorflow

# Global parameters
epochs = 700

Load and format data 

In [65]:
# Train and test directories
train_dir = "coral-classification-deep-learning/coral_img/train/"
test_dir = "coral-classification-deep-learning/coral_img/Test_Mixed/"

# Classes
clases = list(sorted(os.listdir(train_dir)))

# Load data and labels
x_train = np.array([cv2.imread(os.path.join(train_dir, cl, name)) for cl in clases
           for name in os.listdir(os.path.join(train_dir, cl))])

y_train = np.array([n for n, cl in enumerate(clases)
           for name in os.listdir(os.path.join(train_dir, cl))])

y_train = to_categorical(y_train, dtype=int)

print(x_train.shape)
print(y_train.shape)

(620, 256, 256, 3)
(620, 14)


Data augmentation


In [0]:
datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    validation_split=0.2)

Add dense layers

In [0]:
model = vgg16.VGG16(weights='imagenet',include_top=False, input_shape=(256,256,3))

x=model.output
x=GlobalAveragePooling2D()(x)
x=Dense(1024,activation='relu')(x) #we add dense layers so that the model can learn more complex functions and classify for better results.
x=Dense(1024,activation='relu')(x) #dense layer 2
x=Dense(512,activation='relu')(x) #dense layer 3
preds=Dense(14,activation='softmax')(x) #final layer with softmax activation

model=Model(inputs=model.input,outputs=preds)

In [92]:
for i,layer in enumerate(model.layers):
  print(i,layer.name)

0 input_3
1 block1_conv1
2 block1_conv2
3 block1_pool
4 block2_conv1
5 block2_conv2
6 block2_pool
7 block3_conv1
8 block3_conv2
9 block3_conv3
10 block3_pool
11 block4_conv1
12 block4_conv2
13 block4_conv3
14 block4_pool
15 block5_conv1
16 block5_conv2
17 block5_conv3
18 block5_pool
19 global_average_pooling2d_3
20 dense_9
21 dense_10
22 dense_11
23 dense_12


In [0]:
Set trainable layers

In [0]:
for layer in model.layers[:20]:
    layer.trainable=False
for layer in model.layers[20:]:
    layer.trainable=True

Compile and fit model



In [95]:
model.compile(optimizer='Adam',loss='categorical_crossentropy',metrics=['accuracy'])

datagen.fit(x_train)

model.fit_generator(generator=datagen.flow(x_train, y_train, batch_size=60),
                   steps_per_epoch=len(x_train) / 60, epochs=20)



Epoch 1/20
11/10 [===============================] - 12s 1s/step - loss: 2.4555 - acc: 0.2083
Epoch 2/20
11/10 [===============================] - 11s 1s/step - loss: 1.8864 - acc: 0.4477
Epoch 3/20
11/10 [===============================] - 11s 997ms/step - loss: 1.4422 - acc: 0.5353
Epoch 4/20
11/10 [===============================] - 11s 1s/step - loss: 1.1222 - acc: 0.6363
Epoch 5/20
11/10 [===============================] - 11s 1s/step - loss: 0.9938 - acc: 0.6762
Epoch 6/20
11/10 [===============================] - 11s 998ms/step - loss: 0.8167 - acc: 0.7391
Epoch 7/20
11/10 [===============================] - 11s 1s/step - loss: 0.7198 - acc: 0.7647
Epoch 8/20
11/10 [===============================] - 11s 1s/step - loss: 0.5940 - acc: 0.8005
Epoch 9/20
11/10 [===============================] - 11s 1s/step - loss: 0.5635 - acc: 0.8259
Epoch 10/20
11/10 [===============================] - 11s 1s/step - loss: 0.4867 - acc: 0.8439
Epoch 11/20
11/10 [===============================] -

Save model

In [0]:
model.save(experiment + 'model.h5')

Generate .csv

In [108]:
test_files = natsorted(os.listdir(test_dir))

x_test = np.array([cv2.imread(os.path.join(test_dir, name)) 
                   for name in test_files])

predictions_test = model.predict(x_test, verbose=1)

data = pd.DataFrame()
data['Id'] = test_files
data['Category'] = predictions_test.argmax(axis=-1)

data.to_csv(experiment + "envio.csv", index=False)

146/146 [==============================] - 2s 13ms/step


In [117]:
data

,Id,Category
0,aT1.jpg,7
1,aT2.jpg,0
2,aT5.jpg,0
3,aT17.jpg,0
4,aT21.jpg,6
5,aT31.jpg,7
6,aT33.jpg,7
7,aT34.jpg,11
8,aT47.jpg,0
9,aT55.jpg,0
